## LSTM MODEL For Timeseries ##

In [1]:
import numpy as np
import pandas as pd
import ta
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

2023-06-08 22:02:05.441702: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-08 22:02:05.443161: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 22:02:05.472081: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 22:02:05.473170: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 22:02:06.093072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

### Split the data with darts package

In [2]:
#Import your data here
df = pd.read_csv('../raw_data/clean_raw_data.csv', delimiter=",")

In [3]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df

,timestamp,open,high,low,close,volume,CMF,FII,MFI,VWA,...,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-17 04:00:00,10216.35,10238.95,10194.39,10209.45,872.764,-0.120924,-913.266701,28.652105,10185.127432,...,10275.790895,3005.70,2.25,1.70,1,0.45992,0.46011,-1,48.386232,-1
1,2019-09-17 05:00:00,10208.81,10232.07,10204.53,10223.97,819.923,-0.114688,917.954536,36.501645,10187.270366,...,10274.443495,3005.70,2.25,1.70,-1,0.40934,0.45992,-1,42.873022,-1
2,2019-09-17 06:00:00,10224.11,10230.44,10187.88,10224.09,922.804,-0.112046,802.637671,36.306243,10191.939033,...,10273.130448,3005.70,2.25,1.70,-1,0.31251,0.40934,-1,42.291553,-1
3,2019-09-17 07:00:00,10225.02,10230.79,10192.69,10193.85,905.598,-0.124361,-3224.208213,35.631136,10195.919801,...,10271.256418,3005.70,2.25,1.70,-1,0.14629,0.31251,-1,41.595938,-1
4,2019-09-17 08:00:00,10193.21,10215.07,10165.00,10195.82,1089.829,-0.114965,-2456.898021,34.774165,10199.535893,...,10269.459713,3005.70,2.25,1.70,-1,-0.27719,0.14629,-1,41.970769,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32551,2023-06-03 20:00:00,27239.40,27246.40,27101.00,27147.60,15130.376,-0.048901,45306.477416,59.148253,27189.586323,...,27073.676970,4282.37,5.25,5.08,1,0.55827,0.61369,-1,53.765071,1
32552,2023-06-03 21:00:00,27147.70,27173.70,27113.90,27138.80,6229.878,-0.075151,31002.276871,58.559862,27189.451257,...,27073.819644,4282.37,5.25,5.08,-1,0.15512,0.55827,-1,54.075485,1
32553,2023-06-03 22:00:00,27138.90,27153.60,27000.00,27084.90,15389.644,-0.042819,-91926.878625,49.242500,27175.830184,...,27072.988198,4282.37,5.25,5.08,-1,-0.36126,0.15512,-1,44.950464,1
32554,2023-06-03 23:00:00,27084.90,27119.50,27042.00,27052.70,7760.747,-0.045307,-114493.903593,43.575625,27168.759575,...,27071.634000,4282.37,5.25,5.08,-1,-0.86198,-0.36126,-1,45.530993,1


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32556 entries, 0 to 32555
Data columns (total 78 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   timestamp               32556 non-null  datetime64[ns]
 1   open                    32556 non-null  float64       
 2   high                    32556 non-null  float64       
 3   low                     32556 non-null  float64       
 4   close                   32556 non-null  float64       
 5   volume                  32556 non-null  float64       
 6   CMF                     32556 non-null  float64       
 7   FII                     32556 non-null  float64       
 8   MFI                     32556 non-null  float64       
 9   VWA                     32556 non-null  float64       
 10  AOI                     32556 non-null  float64       
 11  KAMA                    32556 non-null  float64       
 12  PPO                     32556 non-null  float6

In [5]:
X = df.drop(['close'], axis=1)
y = df['close'].values.reshape(-1, 1)

In [6]:
X

,timestamp,open,high,low,volume,CMF,FII,MFI,VWA,AOI,...,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-17 04:00:00,10216.35,10238.95,10194.39,872.764,-0.120924,-913.266701,28.652105,10185.127432,-5.729912,...,10275.790895,3005.70,2.25,1.70,1,0.45992,0.46011,-1,48.386232,-1
1,2019-09-17 05:00:00,10208.81,10232.07,10204.53,819.923,-0.114688,917.954536,36.501645,10187.270366,-12.684824,...,10274.443495,3005.70,2.25,1.70,-1,0.40934,0.45992,-1,42.873022,-1
2,2019-09-17 06:00:00,10224.11,10230.44,10187.88,922.804,-0.112046,802.637671,36.306243,10191.939033,-26.571000,...,10273.130448,3005.70,2.25,1.70,-1,0.31251,0.40934,-1,42.291553,-1
3,2019-09-17 07:00:00,10225.02,10230.79,10192.69,905.598,-0.124361,-3224.208213,35.631136,10195.919801,-33.398088,...,10271.256418,3005.70,2.25,1.70,-1,0.14629,0.31251,-1,41.595938,-1
4,2019-09-17 08:00:00,10193.21,10215.07,10165.00,1089.829,-0.114965,-2456.898021,34.774165,10199.535893,-36.033412,...,10269.459713,3005.70,2.25,1.70,-1,-0.27719,0.14629,-1,41.970769,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32551,2023-06-03 20:00:00,27239.40,27246.40,27101.00,15130.376,-0.048901,45306.477416,59.148253,27189.586323,61.621471,...,27073.676970,4282.37,5.25,5.08,1,0.55827,0.61369,-1,53.765071,1
32552,2023-06-03 21:00:00,27147.70,27173.70,27113.90,6229.878,-0.075151,31002.276871,58.559862,27189.451257,58.155882,...,27073.819644,4282.37,5.25,5.08,-1,0.15512,0.55827,-1,54.075485,1
32553,2023-06-03 22:00:00,27138.90,27153.60,27000.00,15389.644,-0.042819,-91926.878625,49.242500,27175.830184,40.827647,...,27072.988198,4282.37,5.25,5.08,-1,-0.36126,0.15512,-1,44.950464,1
32554,2023-06-03 23:00:00,27084.90,27119.50,27042.00,7760.747,-0.045307,-114493.903593,43.575625,27168.759575,11.745882,...,27071.634000,4282.37,5.25,5.08,-1,-0.86198,-0.36126,-1,45.530993,1


In [7]:
y

array([[10209.45],
       [10223.97],
       [10224.09],
       ...,
       [27084.9 ],
       [27052.7 ],
       [26990.  ]])

In [8]:
split_ratio = 0.8
split_index = round(split_ratio * X.shape[0])
X_train = X[:split_index]
X_test = X[split_index:]
# Dropped timestamp column
X_train_notime = X_train.drop('timestamp', axis=1)

X_train_timestamp = X_train[['timestamp']]
X_test_notime = X_test.drop('timestamp', axis=1)
X_test_timestamp = X_test[['timestamp']]


### Scaling, fitting and transforming with RobustScaler

In [9]:
# Initialize the MinMaxScaler
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_notime)
X_test_scaled = scaler.transform(X_test_notime)

In [10]:
X_test_scaled

array([[-0.12297176, -0.12484591, -0.12438619, ..., -1.        ,
         0.60587073, -1.        ],
       [-0.12552907, -0.12719684, -0.12377387, ..., -1.        ,
         0.36148082, -1.        ],
       [-0.12643323, -0.12645569, -0.12357877, ..., -1.        ,
         0.36265575, -1.        ],
       ...,
       [ 0.09551871,  0.09199397,  0.09592626, ..., -1.        ,
        -0.46438057,  0.        ],
       [ 0.09390733,  0.09098304,  0.0971869 , ..., -1.        ,
        -0.42160825,  0.        ],
       [ 0.09294946,  0.08949481,  0.09292473, ..., -1.        ,
        -0.83515551, -1.        ]])

In [11]:
X_train_scaled

array([[-0.40945477, -0.40945809, -0.4084995 , ..., -1.        ,
        -0.2112397 , -1.        ],
       [-0.40967977, -0.40966205, -0.40819514, ..., -1.        ,
        -0.61744255, -1.        ],
       [-0.40922321, -0.40971038, -0.4086949 , ..., -1.        ,
        -0.66028408, -1.        ],
       ...,
       [-0.1224406 , -0.12174198, -0.11935862, ...,  0.        ,
         0.3765217 , -1.        ],
       [-0.12204074, -0.12291299, -0.11973381, ...,  0.        ,
         0.44730709, -1.        ],
       [-0.12230035, -0.12450499, -0.12076334, ...,  0.        ,
         0.40771137, -1.        ]])

In [12]:
X_train_scaled= pd.DataFrame(X_train_scaled, columns=scaler.get_feature_names_out())
X_test_scaled= pd.DataFrame(X_test_scaled, columns=scaler.get_feature_names_out())

In [13]:
X_test_scaled

,open,high,low,volume,CMF,FII,MFI,VWA,AOI,KAMA,...,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,-0.122972,-0.124846,-0.124386,2.563231,-0.698569,-0.495348,-0.296989,-0.123804,0.026819,-0.123658,...,-0.119797,0.056455,3.000000,1.000000,1.0,0.226867,0.295753,-1.0,0.605871,-1.0
1,-0.125529,-0.127197,-0.123774,0.102876,-1.124818,-0.481747,-0.133165,-0.123795,-0.008645,-0.123730,...,-0.119903,0.056455,3.000000,1.000000,0.0,0.080771,0.226867,-1.0,0.361481,-1.0
2,-0.126433,-0.126456,-0.123579,0.145725,-0.968396,-0.355657,0.050758,-0.123810,-0.079015,-0.123775,...,-0.119992,0.056455,3.000000,1.000000,0.0,-0.087594,0.080771,-1.0,0.362656,-1.0
3,-0.125595,-0.126337,-0.123516,-0.102309,-0.795877,-0.282799,0.190750,-0.123802,-0.134714,-0.123817,...,-0.120073,0.041607,3.000000,1.000000,0.0,-0.262186,-0.087594,-1.0,0.394193,-1.0
4,-0.125168,-0.125637,-0.122657,0.085611,-0.772697,-0.146890,0.191375,-0.123777,-0.153268,-0.123827,...,-0.120124,0.041607,3.000000,1.000000,0.0,-0.358133,-0.262186,-1.0,0.446357,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506,0.098518,0.094745,0.098958,0.431085,-0.460950,0.059974,0.371757,0.097939,0.132224,0.095599,...,0.096963,0.387340,6.666667,0.442857,1.0,0.208272,0.233296,-1.0,0.185063,0.0
6507,0.095781,0.092590,0.099345,-0.312795,-0.622375,0.042869,0.349259,0.097935,0.123862,0.095598,...,0.096967,0.387340,6.666667,0.442857,0.0,0.026234,0.208272,-1.0,0.207934,0.0
6508,0.095519,0.091994,0.095926,0.452754,-0.423548,-0.104132,-0.007018,0.097528,0.082054,0.095558,...,0.096942,0.387340,6.666667,0.442857,0.0,-0.206931,0.026234,-1.0,-0.464381,0.0
6509,0.093907,0.090983,0.097187,-0.184849,-0.438846,-0.131119,-0.223707,0.097316,0.011887,0.095479,...,0.096902,0.387340,6.666667,0.442857,0.0,-0.433026,-0.206931,-1.0,-0.421608,0.0


In [14]:
X_test_timestamp= X_test_timestamp.reset_index().drop('index', axis=1)
X_test_timestamp

,timestamp
0,2022-09-05 21:00:00
1,2022-09-05 22:00:00
2,2022-09-05 23:00:00
3,2022-09-06 00:00:00
4,2022-09-06 01:00:00
...,...
6506,2023-06-03 20:00:00
6507,2023-06-03 21:00:00
6508,2023-06-03 22:00:00
6509,2023-06-03 23:00:00


In [15]:
X_train_scaled = pd.concat([X_train_timestamp, X_train_scaled], axis=1)
X_test_scaled = pd.concat([X_test_timestamp, X_test_scaled], axis=1)

In [16]:
X_train_scaled

,timestamp,open,high,low,volume,CMF,FII,MFI,VWA,AOI,...,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-17 04:00:00,-0.409455,-0.409458,-0.408499,-0.760529,-0.903863,0.004703,-0.794351,-0.410291,-0.030278,...,-0.405617,-0.792272,2.666667,-0.160714,1.0,0.163863,0.163949,-1.0,-0.211240,-1.0
1,2019-09-17 05:00:00,-0.409680,-0.409662,-0.408195,-0.764945,-0.865515,0.006893,-0.494201,-0.410227,-0.047058,...,-0.405658,-0.792272,2.666667,-0.160714,0.0,0.141025,0.163863,-1.0,-0.617443,-1.0
2,2019-09-17 06:00:00,-0.409223,-0.409710,-0.408695,-0.756346,-0.849267,0.006755,-0.501673,-0.410087,-0.080562,...,-0.405697,-0.792272,2.666667,-0.160714,0.0,0.097302,0.141025,-1.0,-0.660284,-1.0
3,2019-09-17 07:00:00,-0.409196,-0.409700,-0.408551,-0.757784,-0.925002,0.001940,-0.527488,-0.409968,-0.097034,...,-0.405753,-0.792272,2.666667,-0.160714,0.0,0.022247,0.097302,-1.0,-0.711536,-1.0
4,2019-09-17 08:00:00,-0.410145,-0.410166,-0.409382,-0.742387,-0.867217,0.002858,-0.560256,-0.409860,-0.103392,...,-0.405807,-0.792272,2.666667,-0.160714,0.0,-0.168970,0.022247,-1.0,-0.683919,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26040,2022-09-05 16:00:00,-0.125986,-0.126545,-0.123126,0.554720,-0.201871,-0.293653,-0.680756,-0.123640,-0.190407,...,-0.119593,0.056455,3.000000,1.000000,1.0,-0.819371,-1.196785,-1.0,-0.301367,-1.0
26041,2022-09-05 17:00:00,-0.125535,-0.123370,-0.122774,1.050266,-0.072494,0.148021,-0.309490,-0.124037,-0.171095,...,-0.119624,0.056455,3.000000,1.000000,1.0,-0.407920,-0.819371,0.0,0.331064,-1.0
26042,2022-09-05 18:00:00,-0.122441,-0.121742,-0.119359,3.254262,-0.640331,0.240499,0.202484,-0.123639,-0.096259,...,-0.119646,0.056455,3.000000,1.000000,1.0,-0.053038,-0.407920,0.0,0.376522,-1.0
26043,2022-09-05 19:00:00,-0.122041,-0.122913,-0.119734,0.746357,-0.772577,0.178876,0.143642,-0.123552,-0.013269,...,-0.119671,0.056455,3.000000,1.000000,1.0,0.197155,-0.053038,0.0,0.447307,-1.0


In [17]:
X_train_scaled.shape

(26045, 77)

In [18]:
X_test_scaled

,timestamp,open,high,low,volume,CMF,FII,MFI,VWA,AOI,...,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2022-09-05 21:00:00,-0.122972,-0.124846,-0.124386,2.563231,-0.698569,-0.495348,-0.296989,-0.123804,0.026819,...,-0.119797,0.056455,3.000000,1.000000,1.0,0.226867,0.295753,-1.0,0.605871,-1.0
1,2022-09-05 22:00:00,-0.125529,-0.127197,-0.123774,0.102876,-1.124818,-0.481747,-0.133165,-0.123795,-0.008645,...,-0.119903,0.056455,3.000000,1.000000,0.0,0.080771,0.226867,-1.0,0.361481,-1.0
2,2022-09-05 23:00:00,-0.126433,-0.126456,-0.123579,0.145725,-0.968396,-0.355657,0.050758,-0.123810,-0.079015,...,-0.119992,0.056455,3.000000,1.000000,0.0,-0.087594,0.080771,-1.0,0.362656,-1.0
3,2022-09-06 00:00:00,-0.125595,-0.126337,-0.123516,-0.102309,-0.795877,-0.282799,0.190750,-0.123802,-0.134714,...,-0.120073,0.041607,3.000000,1.000000,0.0,-0.262186,-0.087594,-1.0,0.394193,-1.0
4,2022-09-06 01:00:00,-0.125168,-0.125637,-0.122657,0.085611,-0.772697,-0.146890,0.191375,-0.123777,-0.153268,...,-0.120124,0.041607,3.000000,1.000000,0.0,-0.358133,-0.262186,-1.0,0.446357,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506,2023-06-03 20:00:00,0.098518,0.094745,0.098958,0.431085,-0.460950,0.059974,0.371757,0.097939,0.132224,...,0.096963,0.387340,6.666667,0.442857,1.0,0.208272,0.233296,-1.0,0.185063,0.0
6507,2023-06-03 21:00:00,0.095781,0.092590,0.099345,-0.312795,-0.622375,0.042869,0.349259,0.097935,0.123862,...,0.096967,0.387340,6.666667,0.442857,0.0,0.026234,0.208272,-1.0,0.207934,0.0
6508,2023-06-03 22:00:00,0.095519,0.091994,0.095926,0.452754,-0.423548,-0.104132,-0.007018,0.097528,0.082054,...,0.096942,0.387340,6.666667,0.442857,0.0,-0.206931,0.026234,-1.0,-0.464381,0.0
6509,2023-06-03 23:00:00,0.093907,0.090983,0.097187,-0.184849,-0.438846,-0.131119,-0.223707,0.097316,0.011887,...,0.096902,0.387340,6.666667,0.442857,0.0,-0.433026,-0.206931,-1.0,-0.421608,0.0


In [19]:
X_test_scaled.shape

(6511, 77)

### Reshaping

In [20]:
train_array = X_train_scaled.values
train_array

array([[Timestamp('2019-09-17 04:00:00'), -0.40945477050444384,
        -0.4094580885313184, ..., -1.0, -0.21123970044729934, -1.0],
       [Timestamp('2019-09-17 05:00:00'), -0.4096797661477643,
        -0.4096620531529433, ..., -1.0, -0.6174425478413823, -1.0],
       [Timestamp('2019-09-17 06:00:00'), -0.40922321000150397,
        -0.40971037616649686, ..., -1.0, -0.6602840849758518, -1.0],
       ...,
       [Timestamp('2022-09-05 18:00:00'), -0.12244059995655258,
        -0.12174197645275879, ..., 0.0, 0.3765216958197981, -1.0],
       [Timestamp('2022-09-05 19:00:00'), -0.12204074032518741,
        -0.12291299426586673, ..., 0.0, 0.4473070878169744, -1.0],
       [Timestamp('2022-09-05 20:00:00'), -0.12230035068286475,
        -0.12450498557128183, ..., 0.0, 0.4077113676511389, -1.0]],
      dtype=object)

In [21]:
test_array = X_test_scaled.values
test_array

array([[Timestamp('2022-09-05 21:00:00'), -0.12297175678030636,
        -0.12484591480800945, ..., -1.0, 0.6058707302887394, -1.0],
       [Timestamp('2022-09-05 22:00:00'), -0.125529068004784,
        -0.12719684424040079, ..., -1.0, 0.361480818356735, -1.0],
       [Timestamp('2022-09-05 23:00:00'), -0.12643322821600536,
        -0.12645569372577553, ..., -1.0, 0.36265575028458946, -1.0],
       ...,
       [Timestamp('2023-06-03 22:00:00'), 0.09551870746269368,
        0.09199397355693543, ..., -1.0, -0.46438057322035914, 0.0],
       [Timestamp('2023-06-03 23:00:00'), 0.09390733282883382,
        0.0909830442549866, ..., -1.0, -0.42160825224210297, 0.0],
       [Timestamp('2023-06-04 00:00:00'), 0.09294946012981704,
        0.08949481402161903, ..., -1.0, -0.8351555076708896, -1.0]],
      dtype=object)

In [22]:
# Reshape the data for LSTM input (samples, time steps, features)
n_features = train_array.shape[1]
X_train_reshaped = train_array.reshape((train_array.shape[0], 1, n_features))
X_test_reshaped = test_array.reshape((test_array.shape[0], 1, n_features))

### Defining Model

In [23]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(1, n_features)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

2023-06-08 22:02:08.961925: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-08 22:02:08.962218: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-08 22:02:09.161952: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/conca

In [24]:
model.fit(train_array, y)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Timestamp).

### Evaluation of Model